In [1]:
import pandas as pd
import numpy as np
import os.path as op
import os
from scipy.optimize import minimize
import itertools

from varmodel_utils import objective, get_variance_indices

In [2]:
bp = '/data/RocklandSample/derivatives/paper1/'
exp = 'connectomes_ds_multipipeline'
bpp = op.join(bp, exp)
try:
    os.makedirs(bpp)
except FileExistsError:
    pass

# Load DF and grab graphs
df = pd.read_hdf(bpp + '.h5')
gs = np.array([np.reshape(g, -1) for g in df.graph.values])

# Prune DF to only contain reference, grab graphs
df_ref = df[df['simulation'] == "ref"].reset_index(drop=True)
gs_ref = np.array([np.reshape(g, -1) for g in df_ref.graph.values])

# Reorganize DFs with heirarchical indexing
cols = ["subject", "session", "directions", "pipeline"]
df = df.reset_index(level=0).set_index(cols)
df_ref = df_ref.reset_index(level=0).set_index(cols)

In [3]:
df_touse = df_ref
gs_touse = gs_ref

In [4]:
ind = get_variance_indices(df_touse, mca=False)

cov = np.cov(gs_touse, ddof=1)
cov.shape

/Users/greg/code/gkiar/stability/code/experiments/paper1_analytical_impact/3_analysis/code/exp2_relative_variance/varmodel_utils.py:35: PerformanceWarning: indexing past lexsort depth may impact performance.
  indices = df.loc[query]["index"]


(199, 199)

In [5]:
[len(v[0]) for k, v in ind.items()]

[39402, 1386, 594, 199]

In [6]:
estimate = np.array([np.mean(cov[v]) for k, v in ind.items()])

estimate = np.linalg.solve(np.array(((1,0,0,0),
                                     (1,1,0,0),
                                     (1,1,1,0),
                                     (1,1,1,1))), estimate)
print(estimate)

[ 6.53875161e+03  1.57669033e+01  4.71602677e+00 -2.54114087e+00]


In [7]:
bnds = [(0, None)] * len(estimate)
result = minimize(objective, estimate, (cov, ind), method='L-BFGS-B', options={"disp": True}, bounds=bnds)
print(result)

      fun: 2635.2734297327825
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00677574, 0.        , 0.        , 0.        ])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 90
      nit: 12
   status: 0
  success: True
        x: array([   0.        , 2194.58947273, 2183.53859614, 2178.57383281])
